In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
import psycopg2

# 1. Créer une session Spark
print("Création de la session Spark...")
spark = SparkSession.builder \
    .appName("PostgresToSpark") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.3.9.jar") \
    .getOrCreate()

# 2. Configurer les paramètres de connexion
print("Configuration des paramètres de connexion JDBC pour PostgreSQL...")
jdbc_url = "jdbc:postgresql://postgres:5432/DB_Mastodon"  # Remplacez 'postgres' et 'DB_Mastodon' par vos valeurs
properties = {
    "user": "fadi",        # Votre nom d'utilisateur PostgreSQL
    "password": "fadi",    # Votre mot de passe PostgreSQL
    "driver": "org.postgresql.Driver"  # Driver JDBC PostgreSQL
}

# 3. Lire les données depuis PostgreSQL
print("Lecture des données depuis la table 'Mosotodon_BRONZ' dans PostgreSQL...")
df = spark.read.jdbc(url=jdbc_url, table="Mostodon_BRONZ", properties=properties)
print(f"Nombre de lignes lues : {df.count()}")

# 4. Nettoyer la colonne 'content' des balises HTML
print("Suppression des balises HTML dans la colonne 'content'...")
df_cleaned = df.withColumn("content", regexp_replace(col("content"), "<[^>]+>", ""))

# 5. Filtrer les lignes avec certaines conditions
print("Filtrage des données en fonction des colonnes 'favourites_count', 'reblogs_count', 'replies_count'...")
df_max_values = df_cleaned.filter(
    (col("favourites_count") > 0) & 
    (col("reblogs_count") > 0) & 
    (col("replies_count") > 0)
)
print(f"Nombre de lignes après filtrage : {df_max_values.count()}")

# 6. Afficher les résultats filtrés
df_max_values.show()

# 7. Connexion à PostgreSQL via psycopg2
print("Connexion à PostgreSQL avec psycopg2...")
conn = psycopg2.connect(
    host="postgres",  # Changez ceci si nécessaire
    database="DB_Mastodon",
    user="fadi",
    password="fadi"
)
cursor = conn.cursor()

# 8. Création de la table 'Toots_SILVER' si elle n'existe pas déjà
print("Création de la table 'Mostodon_SILVER' si elle n'existe pas déjà...")
create_table_query = '''
CREATE TABLE IF NOT EXISTS Mostodon_SILVER (
    id BIGINT PRIMARY KEY,
    username TEXT NOT NULL,
    display_name TEXT NOT NULL,
    content TEXT NOT NULL,
    created_at TIMESTAMP NOT NULL,
    favourites_count INT NOT NULL,
    reblogs_count INT NOT NULL,
    replies_count INT NOT NULL
);
'''
cursor.execute(create_table_query)
conn.commit()

# 9. Insertion des données nettoyées dans la table 'Toots_SILVER'
print("Insertion des données nettoyées dans la table 'Mostodon_SILVER' via Spark JDBC...")
df_cleaned.write.jdbc(url=jdbc_url, table="Mostodon_SILVER", mode="append", properties=properties)
print("Insertion terminée.")

# 10. Compter le nombre total d'entrées dans la table 'Toots_SILVER'
print("Calcul du nombre total d'entrées dans la table 'Toots_SILVER'...")
cursor.execute("SELECT COUNT(*) FROM Mostodon_SILVER;")
count = cursor.fetchone()[0]  # Récupérer le résultat du COUNT
print(f"Nombre total d'entrées dans la table 'Mostodon_SILVER' : {count}")

# 11. Fermer le curseur et la connexion PostgreSQL
cursor.close()
conn.close()
print("Connexion à PostgreSQL fermée.")

# 12. Fermer la session Spark
print("Fermeture de la session Spark...")
spark.stop()
print("Session Spark fermée.")


Création de la session Spark...
Configuration des paramètres de connexion JDBC pour PostgreSQL...
Lecture des données depuis la table 'Mosotodon_BRONZ' dans PostgreSQL...
Nombre de lignes lues : 40
Suppression des balises HTML dans la colonne 'content'...
Filtrage des données en fonction des colonnes 'favourites_count', 'reblogs_count', 'replies_count'...
Nombre de lignes après filtrage : 1
+------------------+---------------+---------------+-------------------------------------+--------------------+----------------+-------------+-------------+
|                id|       username|   display_name|                              content|          created_at|favourites_count|reblogs_count|replies_count|
+------------------+---------------+---------------+-------------------------------------+--------------------+----------------+-------------+-------------+
|113236821864965313|sotonogi_nonono|そとのぎ ののの|影の地に行く条件すら分かってないけ...|2024-10-02 08:12:...|               0|            2|            0|
+-